# AWS Machine Learning Purpose-built Accelerators Tutorial
## Learn how to use [AWS Trainium](https://aws.amazon.com/machine-learning/trainium/) and [AWS Inferentia](https://aws.amazon.com/machine-learning/inferentia/) with [Amazon SageMaker](https://aws.amazon.com/sagemaker/), to optimize your ML workload
## Part 3/3 - Compiling and deploying a Bert model to AWS Inferentia 2 with SageMaker + [Hugging Face Optimum Neuron](https://huggingface.co/docs/optimum-neuron/index)

**SageMaker studio Kernel: PyTorch 1.13 Python 3.9 CPU - ml.t3.medium** 

In this tutorial, you'll learn how to compile a model to AWS Inferentia and then deploy it to a SageMaker real-time endpoint powered by AWS Inferentia2. First we'll kick-off a SageMaker job to compile the model. We need to do this once. After that, we can deploy our model to a SageMaker endpoint and finally get some predictions.

In section 02, you extract some metadata from the Optimum Neuron API and render a table with the current tested/supported models (similar models not listed there can also be compatible, but you need to check by yourself). This table is important for you to understand which models can be selected for deployment. However, if you also need to fine-tune your model, check a similar table in the notebook **Part 2** to see which models can be fine-tuned with AWS Trainium using HF Optimum Neuron. That way you can plan your end2end solution and start implementing it right now.

## 1) Install some required packages

In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install -U sagemaker

## 2) Supported models/tasks

Models with **[TP]** after the name support Tensor Parallelism

In [ ]:
from IPython.display import Markdown, display

display(Markdown("../docs/optimum_neuron_models.md"))

## 3) Compiling a pre-trained model to AWS Inferentia

**IMPORTANT:** Copy the **SageMaker training job name** from the previous notebook **02_ModelFineTuning** or from your AWS Console/SageMaker and set the variable **training_job_name**. It is necessary because we'll use the fine-tuned model as the input for the compilation job.

In [ ]:
import os
import boto3
import sagemaker

print(sagemaker.__version__)
if not sagemaker.__version__ >= "2.146.0": print("You need to upgrade or restart the kernel if you already upgraded")

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = sess.boto_region_name

training_job_name=""

if os.path.isfile("training_job_name.txt"): training_job_name = open("training_job_name.txt", "r").read().strip()
assert len(training_job_name)>0, "Please copy the name of the training_job you ran in the previous notebook and set training_job_name"
checkpoint_s3_uri=f"s3://{bucket}/output/{training_job_name}/output/model.tar.gz"

if not os.path.isdir('src'): os.makedirs('src', exist_ok=True)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")
print(f"Training job name: {training_job_name}")
print(f"Model S3 URI: {checkpoint_s3_uri}")

### 3.1) Compilation script that will be invoked by SageMaker

In [ ]:
!pygmentize src/compile.py

In [ ]:
!pygmentize src/requirements.txt

### 3.2) SageMaker Estimator
This object will help you to configure the compilation job (SageMaker Training Job).

This job will invoke **compile.py** script, which will compile our model to Inferentia2 and than save the artifacts for deployment.

In [ ]:
task="SequenceClassification"
# Source: https://huggingface.co/docs/optimum-neuron/guides/export_model#exporting-a-model-to-neuron-via-neuronmodel
input_shapes={"batch_size": 1, "sequence_length": 512}

In [ ]:
import json
import logging
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="compile.py", # Specify your train script
    source_dir="src",
    role=role,
    sagemaker_session=sess,
    container_log_level=logging.DEBUG,
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    output_path=f"s3://{bucket}/output",
    disable_profiler=True,
    
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.18.0-ubuntu20.04",
    
    volume_size = 512,
    hyperparameters={     
        "task": task,
        "input_shapes": f"'{json.dumps(input_shapes)}'",
        "dynamic_batch_size": True
    }
)
estimator.framework_version = '1.13.1' # workround when using image_uri

In [ ]:
estimator.fit({"checkpoint": checkpoint_s3_uri})

## 4) Deploy a SageMaker real-time endpoint

In [ ]:
import logging
from sagemaker.utils import name_from_base
from sagemaker.pytorch.model import PyTorchModel

# depending on the inf2 instance you deploy the model you'll have more or less accelerators
# we'll ask SageMaker to launch 1 worker per core

model_data=estimator.model_data
print(f"Model data: {model_data}")

instance_type_idx=0 # default ml.inf2.xlarge
instance_types=['ml.inf2.xlarge', 'ml.inf2.8xlarge', 'ml.inf2.24xlarge','ml.inf2.48xlarge']
num_workers=[2,2,12,24]

print(f"Instance type: {instance_types[instance_type_idx]}. Num SM workers: {num_workers[instance_type_idx]}")
pytorch_model = PyTorchModel(
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference-neuronx:1.13.1-neuronx-py310-sdk2.18.0-ubuntu20.04",
    model_data=model_data,
    role=role,    
    name=name_from_base('bert-spam-classifier'),
    sagemaker_session=sess,
    container_log_level=logging.DEBUG,
    model_server_workers=num_workers[instance_type_idx], # 1 worker per inferentia chip
    framework_version="1.13.1",
    env = {
        'SAGEMAKER_MODEL_SERVER_TIMEOUT': '3600',
        'TASK': task
    }
    # for production it is important to define vpc_config and use a vpc_endpoint
    #vpc_config={
    #    'Subnets': ['<SUBNET1>', '<SUBNET2>'],
    #    'SecurityGroupIds': ['<SECURITYGROUP1>', '<DEFAULTSECURITYGROUP>']
    #}
)
pytorch_model._is_compiled_model = True

In [ ]:
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_types[instance_type_idx],
    model_data_download_timeout=3600, # it takes some time to download all the artifacts and load the model
    container_startup_health_check_timeout=1800
)

## 5) Run a simple test

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [65]:
import time

labels={0: "not spam", 1: "spam"}
not_spam=" Deezer.com 10,406,168 Artist DB\n\nWe have scraped the Deezer Artist DB, right now there are 10,406,168 listings according to Deezer.com\n\nPlease note in going through part of the list, it is obvious there are mistakes inside their system.\n\nExamples include and Artist with &amp; in its name might also be found with "and" but the Albums for each have different totals etc. Have no clue if there are duplicate albums etc do this error in their system. Even a comma in a name could mean the Artist shows up more than once, I saw in 1 instance that 1 Artist had 6 different ArtistIDs due to spelling errors.\n\nSo what is this DB, very simple, it gives you the ArtistID and the actual name of the Artist in another column. If you want to see the artist you add the baseurl to the ArtistID\n\nAn example is ArtistID 115 is AC/DC\n\n[https://www.deezer.com/us/artist/115](https://www.deezer.com/us/artist/115)\n\nYou do not have to use [https://www.deezer.com/us/artist/](https://www.deezer.com/us/artist/) if your first language is other than English, just see if Deezer supports your language and use that baseref\n\nFrench for example is [https://www.deezer.com/fr/artist/115](https://www.deezer.com/fr/artist/115)\n\nI am providing the DB in 3 different formats:\n\n \n\nI tried posting download links here but it seems Reddit does not like that so get them here:\n\n[https://pastebin\\[DOT\\]com/V3KJbgif](https://pastebin.com/V3KJbgif)\n\n&amp;#x200B;\n\n**Special thanks go to** [**/user/KoalaBear84**](https://www.reddit.com/user/KoalaBear84) **for writing the scraper.**\n\n&amp;#x200B;\n\n**Cross Posted to related Reddit Groups**"
spam="🚨 ATTENTION ALL USERS! 🚨\n\n🆘 Are you looking for a way to GET RICH QUICK? 🆘\n\n💰 Don't waste your time with boring old jobs! 💰\n\n💸 Join our CRAZY MONEY-MAKING SYSTEM today! 💸\n\n🤑 Just sign up and start earning BIG BUCKS right away! 🤑\n\n👉 Plus, if you refer your friends, you'll get even MORE CASH! 👈\n\n🔥 This is the HOTTEST OFFER of the year! 🔥\n\n👍 Don't wait"

for i,text in enumerate([not_spam, spam]):
    t=time.time()
    pred = predictor.predict({"prompt": text})
    elapsed = (time.time()-t)*1000
    print(f"Elapsed time: {elapsed}")
    print(f"Pred: {i} - {labels[pred[0][0]]} / score: {pred[0][1]}")

Elapsed time: 30.43961524963379
Pred: 0 - not spam / score: 5.735881805419922
Elapsed time: 28.235435485839844
Pred: 1 - spam / score: 5.159011363983154


## 5) Cleanup

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()